<a href="https://colab.research.google.com/github/engbrunoconsultor/WeatherForecast/blob/main/WeatherForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd

Buscando o código das cidades para ser usado na API de previsão do tempo

In [2]:
url_ibge = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"

response_ibge = requests.get(url_ibge)

if response_ibge.status_code==200:
  municipios = response_ibge.json() #convertendo para JSON

  #Transformando em DataFrame
  df_municipios = pd.DataFrame(municipios)
  display(df_municipios)


else:
  print("Erro ao acessar API do IBGE:", response_ibge.status_code)

,id,nome,microrregiao,regiao-imediata
0,1100015,Alta Floresta D'Oeste,"{'id': 11006, 'nome': 'Cacoal', 'mesorregiao':...","{'id': 110005, 'nome': 'Cacoal', 'regiao-inter..."
1,1100023,Ariquemes,"{'id': 11003, 'nome': 'Ariquemes', 'mesorregia...","{'id': 110002, 'nome': 'Ariquemes', 'regiao-in..."
2,1100031,Cabixi,"{'id': 11008, 'nome': 'Colorado do Oeste', 'me...","{'id': 110006, 'nome': 'Vilhena', 'regiao-inte..."
3,1100049,Cacoal,"{'id': 11006, 'nome': 'Cacoal', 'mesorregiao':...","{'id': 110005, 'nome': 'Cacoal', 'regiao-inter..."
4,1100056,Cerejeiras,"{'id': 11008, 'nome': 'Colorado do Oeste', 'me...","{'id': 110006, 'nome': 'Vilhena', 'regiao-inte..."
...,...,...,...,...
5565,5222005,Vianópolis,"{'id': 52016, 'nome': 'Pires do Rio', 'mesorre...","{'id': 520002, 'nome': 'Anápolis', 'regiao-int..."
5566,5222054,Vicentinópolis,"{'id': 52015, 'nome': 'Meia Ponte', 'mesorregi...","{'id': 520009, 'nome': 'Piracanjuba', 'regiao-..."
5567,5222203,Vila Boa,"{'id': 52012, 'nome': 'Entorno de Brasília', '...","{'id': 520022, 'nome': 'Flores de Goiás', 'reg..."
5568,5222302,Vila Propício,"{'id': 52012, 'nome': 'Entorno de Brasília', '...","{'id': 520018, 'nome': 'Ceres - Rialma - Goian..."


Ajustando os dados para mostrar apenas a informação de interesse

In [3]:
df_municipios["codigo_ibge"] = df_municipios["id"]
df_municipios["municipio"] = df_municipios["nome"]
df_municipios["uf"] = df_municipios["microrregiao"].apply(lambda x: x["mesorregiao"]["UF"]["sigla"])

df_municipios = df_municipios[["codigo_ibge", "municipio", "uf"]]

display(df_municipios)

,codigo_ibge,municipio,uf
0,1100015,Alta Floresta D'Oeste,RO
1,1100023,Ariquemes,RO
2,1100031,Cabixi,RO
3,1100049,Cacoal,RO
4,1100056,Cerejeiras,RO
...,...,...,...
5565,5222005,Vianópolis,GO
5566,5222054,Vicentinópolis,GO
5567,5222203,Vila Boa,GO
5568,5222302,Vila Propício,GO


Insira o nome do município em "municipio_desejado"

In [4]:
municipio_desejado = "Campo Grande"

municipio = df_municipios[df_municipios["municipio"] == municipio_desejado]

display(municipio)

,codigo_ibge,municipio,uf
1087,2401305,Campo Grande,RN
1665,2701506,Campo Grande,AL
5122,5002704,Campo Grande,MS


Agora vamos buscar os dados de previsão no INMET baseado no código

In [6]:
url_inmet = "https://apiprevmet3.inmet.gov.br/previsao/5002704"
response_inmet = requests.get(url_inmet)

if response_inmet.status_code == 200:
  previsao = response_inmet.json()

  df_previsao_completa = pd.DataFrame(previsao)
  display(df_previsao)

else:
  print("Erro ao buscar a API do INMET: ", response_inmet.status_code)

,5002704
26/02/2025,"{'manha': {'uf': 'MS', 'entidade': 'Campo Gran..."
27/02/2025,"{'manha': {'uf': 'MS', 'entidade': 'Campo Gran..."
28/02/2025,"{'uf': 'MS', 'entidade': 'Campo Grande', 'resu..."
01/03/2025,"{'uf': 'MS', 'entidade': 'Campo Grande', 'resu..."
02/03/2025,"{'uf': 'MS', 'entidade': 'Campo Grande', 'resu..."


A intenção desse projeto é obter apenas a previsão do tempo identificando quando irá chover, dessa forma, vamos precisar fazer uma filtragem dos dados

In [7]:
data_previsao = list(previsao.keys())[0]

periodos = previsao[data_previsao]

dados = []
for periodo, info in periodos.items():
  if isinstance(info, dict) and "resumo" in info:
    dados.append({
        "Data": data_previsao,
        "Período": periodo.capitalize(),
        "Previsão": info["resumo"]
    })

df_previsao = pd.DataFrame(dados)

display(df_previsao)

,Data,Período,Previsão
0,5002704,28/02/2025,Muitas nuvens com pancadas de chuva isoladas
1,5002704,01/03/2025,Muitas nuvens com pancadas de chuva isoladas
2,5002704,02/03/2025,Muitas nuvens com possibilidade de chuva isolada
